In [1]:
import pandas as pd
import glob
import statsmodels.api as sm

In [2]:
metric = "ndcg_cut_10" # map # ndcg_cut_10
result_format = "ndcgeval" # treceval # ndcgeval
results_from = "LLMJudge2024"

In [3]:
results_df_list = []

for infile in glob.glob(f'./results/{results_from}/*.{result_format}'):
    judger = infile.split('/')[3].split('.')[2]
    result_df = pd.read_csv(infile, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])
    result_df = result_df[result_df['qid'] != 'all']
    result_df['metric'] = result_df['metric'].apply(lambda x: x.rstrip())
    result_df = result_df[(result_df['metric'] == metric)]
    result_df.drop(['metric'], axis=1, inplace=True)
    result_df['judged_by'] = judger
    result_df['qid'] = result_df['qid'].astype(int)
    result_df['score'] = result_df['score'].astype(float)
    results_df_list.append(result_df)
 
results_dfs = pd.concat(results_df_list)
results_dfs

,run_id,qid,score,judged_by
2,naverloo_fs_RR_duo,2002168,1.0000,TREMA-rubric0
12,naverloo_fs_RR_duo,2004282,0.3396,TREMA-rubric0
22,naverloo_fs_RR_duo,2004980,0.4221,TREMA-rubric0
32,naverloo_fs_RR_duo,2005952,0.6944,TREMA-rubric0
42,naverloo_fs_RR_duo,2007816,0.6512,TREMA-rubric0
...,...,...,...,...
9042,naverloo_bm25_splades_RR,3100119,0.8665,Olz-exp
9052,naverloo_bm25_splades_RR,3100235,0.6217,Olz-exp
9062,naverloo_bm25_splades_RR,3100289,0.7046,Olz-exp
9072,naverloo_bm25_splades_RR,3100399,0.5089,Olz-exp


In [5]:
# queries_judged = set(results_dfs['qid'])
# real_queries_judged = [x for x in queries_judged if x < 3000000]
# t5_queries_judged = [x for x in queries_judged if x > 3000000 and x < 3100000]
# gpt4_queries_judged = [x for x in queries_judged if x > 3100000]

# print(len(real_queries_judged))
# print(len(t5_queries_judged))
# print(len(gpt4_queries_judged))

In [4]:
qid_to_info = pd.read_csv("infos/query_to_info.txt", sep='\t')
doc_to_info = pd.read_csv("infos/doc_to_info.txt", sep='\t')
model_to_info = pd.read_csv("infos/model_to_info.txt", sep='\t')

In [5]:
data = pd.merge(results_dfs, qid_to_info, on='qid')
# data = pd.merge(data, doc_to_info, on='qid')
data = pd.merge(data, model_to_info, on='run_id')

In [7]:
# These are the features that do not seem to be very useful, relevant, or concrete.
# QL is a binary feature and we have QW, so it should be dropped.
data.drop(['QL'], axis=1, inplace=True)
data.drop(['isGPT4'], axis=1, inplace=True)
data.drop(['QDR'], axis=1, inplace=True)
data.drop(['QDS'], axis=1, inplace=True)
data.drop(['isLLM'], axis=1, inplace=True)

In [8]:
# def QD_value(row):
#     if row['judged_by'] == 'nist':
#         return row['QDR']
#     elif row['judged_by'] == 'gpt4':
#         return row['QDS']

In [9]:
# data['QD'] = data.apply(QD_value, axis=1)

In [8]:
data['QT'] = data['QT'].astype('category') 
data['QT'] = data['QT'].replace({0: 'Human', 1: 'T5', 2: 'GPT4'}) 

In [ ]:
data

In [33]:
mixed_model = "score ~ QD + QW + APL + pipeline + C(judged_by, Treatment(reference='nist')) +  QT + C(ST, Treatment(reference='Other')) + QT * C(ST, Treatment(reference='Other')) + C(ST, Treatment(reference='Other')) * C(judged_by, Treatment(reference='nist'))  "

In [ ]:
model = sm.MixedLM.from_formula(mixed_model, data, groups=data["qid"])
result = model.fit()
result.summary()

In [25]:
data2 = data.drop(['QD'], axis=1)

In [ ]:
score_diff_data = data2.pivot(index=['run_id', 'qid', 'QW', 'APL', 'QT', 'ST', 'pipeline'], columns=['judged_by'], values='score')
score_diff_data.columns.name = None
score_diff_data = score_diff_data.reset_index()
score_diff_data['score_diff'] = score_diff_data['gpt4'] - score_diff_data['nist']
score_diff_data

In [35]:
score_diff_mixed_model = "score_diff ~  QW + APL + pipeline  +  QT + C(ST, Treatment(reference='Other')) + QT * C(ST, Treatment(reference='Other')) "

In [ ]:
model = sm.MixedLM.from_formula(score_diff_mixed_model, score_diff_data, groups=score_diff_data["qid"])
result = model.fit()
result.summary()

In [11]:
real_queries_real_judgments = data[((data['Judge'] == 'nist') & (data['qid'].isin(real_queries_judged)))]
synthetic_queries_real_judgments = data[((data['Judge'] == 'nist') & (data['qid'].isin(t5_queries_judged) | data['qid'].isin(gpt4_queries_judged)))]

In [12]:
real_queries_synthetic_judgments = data[((data['Judge'] == 'gpt4') & (data['qid'].isin(real_queries_judged)))]
synthetic_queries_synthetic_judgments = data[((data['Judge'] == 'gpt4') & (data['qid'].isin(t5_queries_judged) | data['qid'].isin(gpt4_queries_judged)))]

In [13]:
mixed_model_condition_Qreal = "score ~ QL + QDR + QW + C(LLM, Treatment(reference='Other')) + pipeline"
mixed_model_condition_Qsynthetic = "score ~ QL + QDS + QW + C(LLM, Treatment(reference='Other')) + pipeline"

In [ ]:
model = sm.MixedLM.from_formula(mixed_model_condition_Qreal, real_queries_real_judgments, groups=real_queries_real_judgments["run_id"])
result = model.fit()
result.summary()

In [ ]:
model = sm.MixedLM.from_formula(mixed_model_condition_Qsynthetic, synthetic_queries_real_judgments, groups=synthetic_queries_real_judgments["run_id"])
result = model.fit()
result.summary()

In [ ]:
model = sm.MixedLM.from_formula(mixed_model_condition_Qreal, real_queries_synthetic_judgments, groups=real_queries_synthetic_judgments["run_id"])
result = model.fit()
result.summary()

In [ ]:
model = sm.MixedLM.from_formula(mixed_model_condition_Qsynthetic, synthetic_queries_synthetic_judgments, groups=synthetic_queries_synthetic_judgments["run_id"])
result = model.fit()
result.summary()

In [18]:
# Merging the dataframes on 'run_id' and 'qid'
real_queries_diff = pd.merge(real_queries_real_judgments, real_queries_synthetic_judgments[['run_id', 'qid', 'score']], on=['run_id', 'qid'], suffixes=('_a', '_b'))
# Subtracting the 'Score' values
real_queries_diff['score_ab'] = real_queries_diff['score_a'] - real_queries_diff['score_b']
real_queries_diff['score_ba'] = real_queries_diff['score_b'] - real_queries_diff['score_a']

In [ ]:
real_queries_diff

In [ ]:
model = sm.MixedLM.from_formula("score_ab ~ QL + QDR + QW + C(LLM, Treatment(reference='Other')) + pipeline", real_queries_diff, groups=real_queries_diff["run_id"])
result = model.fit()
result.summary()

In [ ]:
model = sm.MixedLM.from_formula("score_ba ~ QL + QDR + QW + C(LLM, Treatment(reference='Other')) + pipeline", real_queries_diff, groups=real_queries_diff["run_id"])
result = model.fit()
result.summary()

In [21]:
# Merging the dataframes on 'run_id' and 'qid'
synthetic_queries_diff = pd.merge(synthetic_queries_real_judgments, synthetic_queries_synthetic_judgments[['run_id', 'qid', 'score']], on=['run_id', 'qid'], suffixes=('_a', '_b'))
# Subtracting the 'Score' values
synthetic_queries_diff['score_ab'] = synthetic_queries_diff['score_a'] - synthetic_queries_diff['score_b']
synthetic_queries_diff['score_ba'] = synthetic_queries_diff['score_b'] - synthetic_queries_diff['score_a']

In [ ]:
synthetic_queries_diff

In [ ]:
model = sm.MixedLM.from_formula("score_ab ~ QL + QDR + QW + C(LLM, Treatment(reference='Other')) + pipeline", synthetic_queries_diff, groups=synthetic_queries_diff["run_id"])
result = model.fit()
result.summary()

In [ ]:
model = sm.MixedLM.from_formula("score_ba ~ QL + QDR + QW + C(LLM, Treatment(reference='Other')) + pipeline", synthetic_queries_diff, groups=synthetic_queries_diff["run_id"])
result = model.fit()
result.summary()

## Extra Experiments (refer to: "Extra Exp. 1")

In [6]:
def get_result(result_file):
    result_df = pd.read_csv(result_file, sep='\t', header=None, names=['run_id', 'metric', 'qid', 'score'])
    result_df = result_df[result_df['qid'] != 'all']
    result_df['score'] = result_df['score'].astype(float)
    result_df['qid'] = result_df['qid'].astype(int)
    result_df['metric'] = result_df['metric'].apply(lambda x: x.rstrip())
    result_df = result_df[(result_df['metric'] == metric)]
    result_df.drop(['metric'], axis=1, inplace=True)
    return result_df

In [7]:
real_judge_results = get_result(result_file="results/all.pass.nist.ndcgeval")
synthetic_judge_results = get_result(result_file="results/all.pass.gpt4.ndcgeval")

In [9]:
# Merging the dataframes on 'run_id' and 'qid'
results_diff = pd.merge(real_judge_results, synthetic_judge_results, on=['run_id', 'qid'], suffixes=('_real', '_synthetic'))
# Subtracting the 'Score' values
results_diff['score_RS'] = results_diff['score_real'] - results_diff['score_synthetic']
results_diff['score_SR'] = results_diff['score_synthetic'] - results_diff['score_real']

In [13]:
results_diff = pd.merge(results_diff, qid_to_info, on='qid')
results_diff = pd.merge(results_diff, model_to_info, on='run_id')

In [ ]:
results_diff

Selected: score_SR

In [17]:
results_diff['qd_RS'] = results_diff['QDR'] - results_diff['QDS']
results_diff['qd_SR'] = results_diff['QDS'] - results_diff['QDR']

In [21]:
mixed_model_1 = "score_SR ~ Synthetic + QL + qd_SR + QW + DL + pipeline + C(LLM, Treatment(reference='Other')) + Synthetic * C(LLM, Treatment(reference='Other'))"

In [ ]:
model = sm.MixedLM.from_formula(mixed_model_1, results_diff, groups=results_diff["run_id"])
result = model.fit()
result.summary()